In [1]:
!pip freeze > requirements.txt. 

In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [ ]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [ ]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

In [ ]:
# pd.show_versions()

## Static Examples

In [ ]:
from nba_api.stats.static import players
# Find players by full name.
players.find_players_by_full_name('james')

# Find players by first name.
players.find_players_by_first_name('lebron')

# Find players by last name.
players.find_players_by_last_name('^(james|love)$')

# Get all players.
# players.get_players()

## Begin Database Connection (sqlite)

In [ ]:
# Base = declarative_base()
# engine = create_engine('sqlite:///db.sqlite', echo=False)
# conn = engine.connect()
# Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# # Create a Session Object to Connect to DB
# # ----------------------------------
# session = Session(bind=engine)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

## List of all players
find specific player

In [ ]:
from nba_api.stats.static import players
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print('Number of players fetched: {}'.format(len(nba_players)))
nba_players[:]
all_players = pd.DataFrame(nba_players)

### Find specific player

In [ ]:
## loop to find player
player_name = [player for player in nba_players
                   if player['full_name'] == 'Tim Duncan'][0]
player_name

## Get Career Stats by player id number
Get player_id number from get_players above

#### Key: 

* GP: Games Played
* MIN: Minutes Played
* FGM: Field Goals Made
* FGA: Field Goals Attempted
* FG%: Field Goal Percentage
* 3PM: 3 Point Field Goals Made
* 3PA: 3 Point Field Goals Attempted
* 3P%: 3 Point Field Goals Percentage
* FTM: Free Throws Made
* FTA: Free Throws Attempted
* FT%: Free Throw Percentage
* OREB: Offensive Rebounds
* DREB: Defensive Rebounds
* REB: Rebounds
* AST: Assists
* TOV: Turnovers
* STL: Steals
* BLK: Blocks
* PF: Personal Fouls
* DD2: Double Doubles
* TD3: Trible Doubles
* PTS: Points




example

In [ ]:
# Display all columns
pd.set_option('display.max_columns', 500)
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='203076')
career_df = career.get_data_frames()[0]

career_dict = career_df.to_dict('records')
first_5_years = career_df[0:5]

In [ ]:
career_df.columns

-----------

#### PySpark databases 
testing

In [ ]:
# import os
# import findspark
# findspark.init()

In [ ]:
# # Start Spark session
# from pyspark.sql import SparkSession
# from pyspark import SparkFiles
# spark = SparkSession.builder.appName("nbaStats").getOrCreate()

In [ ]:
# with open("sample.json", "w") as outfile:  
#     json.dump(career_dict, outfile) 

In [ ]:
# sc = spark.sparkContext
# path = "./sample.json"
# players_json_spark = spark.read.json(path)

In [ ]:
# players_json_spark.printSchema()

In [ ]:
# Creates a temporary view using the DataFrame
# players_json_spark.createOrReplaceTempView("people")

In [ ]:
# players_json_spark.show()

In [ ]:
# # playersDF_spark = spark.range(3).toDF("myCol")
# newRow = players_json_spark
# appended = players_json_spark.union(newRow)
# display(appended)

In [ ]:
# appended.count()

In [ ]:
# def customFunction(row):

#    return (row)

# sample2 = appended.rdd.map(customFunction)
# sample2

In [ ]:
# for ids in all_players_ids: 
#     player_to_pass = ids

In [ ]:
# for f in appended.collect(): 
#         print (f)

In [ ]:
# playersDF_spark.withColumn('newprice', dataframe]).show()

In [ ]:
# otherPeople = spark.read.json(path)
# otherPeople.show()
# otherPeople.select(otherPeople["AST"]).show()

In [ ]:
# Create DataFrame manually
# dataframe = spark.createDataFrame(path, schema=main_df_columns)
# dataframe.show()

------------

## Get first 5 years of each player id.

In [ ]:
all_players_ids = list(all_players['id'].values.astype(str))
len(all_players_ids)

In [ ]:
all_players_col = ['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS']
len(all_players_col)

In [ ]:
main_df = pd.DataFrame(columns=all_players_col)

pbar = tqdm(total=len(all_players_ids))
try: 
    for ids in all_players_ids: 
        player_to_pass = ids

        career = playercareerstats.PlayerCareerStats(player_id=player_to_pass)
        career_df = career.get_data_frames()[0]
        first_5_years = career_df[0:5]
        main_df = main_df.append(first_5_years, ignore_index = True)
        pbar.update(1)
    pbar.close()
except:
    print("something happened")

In [ ]:
all_players_ids

In [ ]:
five_year_all_players = main_df.sort_values(by='SEASON_ID', ascending=False).copy()
five_year_all_players = five_year_all_players.reset_index(drop=True)
five_year_all_players

In [ ]:
five_year_all_players_to_json = five_year_all_players.to_json(orient='records')

In [ ]:
# SAVE: Player_position
with open(f'./_players_all_data.json', 'w') as fp:
    json.dump(five_year_all_players_to_json, fp)

-------------

# Open Saved JSON

In [ ]:
with open('./_players_all_data.json') as json_file:
    data = json.load(json_file)

In [ ]:
five_year_all_players = pd.read_json(data)
twenty_years_all_players = five_year_all_players[five_year_all_players['SEASON_ID'] > '1999-98']
twenty_years_all_players

In [ ]:
import plotly.express as px


fig = px.scatter(twenty_years_all_players, x="PTS", y="SEASON_ID", log_x=True, hover_name="PLAYER_ID", hover_data=["SEASON_ID", "PTS"])

fig.show()

In [ ]:
# five_year_all_players.set_index('SEASON_ID')

In [ ]:
## Save all players data to sqlite db 'all_players'
# all_players.to_sql('all_players', con=engine)

## Retreive all players from sqlite db
# engine.execute("SELECT * FROM all_players").fetchall()